In [44]:
from nltk.corpus import brown
import nltk
import numpy as np
import pandas as pd

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")
nltk.download('omw-1.4')
nltk.download('brown')

[nltk_data] Downloading package stopwords to /Users/tatsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tatsu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /Users/tatsu/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tatsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tatsu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package brown to /Users/tatsu/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [45]:
import re

def cleaning_word(text):
    # 記号文字の消去
    pattern1 = r'\n'
    text = re.sub(pattern1, ' ', text)    
    pattern2 = r'<.*>|[|]'
    text = re.sub(pattern2, '', text)    
    pattern3 = r'\(|\)'
    text = re.sub(pattern3, '', text)
    pattern4 = r'|:|;|\+|-|\?|/|!|~|='
    text = re.sub(pattern4, '', text)
    pattern5 = r'`|\'|$|&|#'
    text = re.sub(pattern5, '', text)
    return text

a = "a\nb"
print(cleaning_word(a))

a b


In [46]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma

def tokenize_text(text):
  text = re.sub('[.,]', '', text)
  return text.split()

en_stop = nltk.corpus.stopwords.words('english')
def remove_stopwords(word, stopwordset):
  if word in stopwordset:
    return None
  else:
    return word

def preprocessing_text(text):
  text = cleaning_word(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  return tokens


In [47]:
corpus_paper = pd.read_csv('paper_info.csv', header=None)
corpus_paper = corpus_paper.set_axis(['field', 'abst'], axis=1)
corpus_paper = corpus_paper.to_dict(orient='records')
for i, paper_dict in enumerate(corpus_paper):
    corpus_paper[i]["abst"] = preprocessing_text(paper_dict["abst"])

corpus_paper

[{'field': 'cs.AI',
  'abst': ['explainability',
   'key',
   'challenge',
   'major',
   'research',
   'theme',
   'ai',
   'research',
   'developing',
   'intelligent',
   'system',
   'capable',
   'working',
   'humans',
   'effectively',
   'obvious',
   'choice',
   'developing',
   'explainable',
   'intelligent',
   'system',
   'rely',
   'employ',
   'knowledge',
   'representation',
   'formalism',
   'inherently',
   'tailor',
   'towards',
   'express',
   'human',
   'knowledge',
   'eg',
   'interrogative',
   'agenda',
   'scope',
   'work',
   'focus',
   'formal',
   'concept',
   'analysis',
   'fca',
   'standard',
   'knowledge',
   'representation',
   'formalism',
   'express',
   'interrogative',
   'agenda',
   'particular',
   'categorize',
   'object',
   'wrt',
   'given',
   'set',
   'feature',
   'several',
   'fcabased',
   'algorithm',
   'already',
   'use',
   'standard',
   'machine',
   'learning',
   'task',
   'classification',
   'outlier',
   

In [48]:
import gensim
from gensim import corpora

doc_list = [dict["abst"] for dict in corpus_paper]

#辞書の作成
dictionary = corpora.Dictionary(doc_list)
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in doc_list]

#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'accuracy': 0, 'agenda': 1, 'ai': 2, 'algorithm': 3, 'already': 4, 'analysis': 5, 'application': 6, 'assign': 7, 'beforehand': 8, 'call': 9, 'capable': 10, 'categorization': 11, 'categorize': 12, 'challenge': 13, 'choice': 14, 'classification': 15, 'complexity': 16, 'concept': 17, 'construct': 18, 'correct': 19, 'data': 20, 'detection': 21, 'developing': 22, 'different': 23, 'effectively': 24, 'eg': 25, 'employ': 26, 'exist': 27, 'explain': 28, 'explainability': 29, 'explainable': 30, 'express': 31, 'fca': 32, 'fcabased': 33, 'feature': 34, 'fix': 35, 'focus': 36, 'formal': 37, 'formalism': 38, 'given': 39, 'good': 40, 'hence': 41, 'human': 42, 'humans': 43, 'importance': 44, 'increase': 45, 'inherently': 46, 'intelligent': 47, 'interrogative': 48, 'iteratively': 49, 'key': 50, 'know': 51, 'knowledge': 52, 'lattice': 53, 'learning': 54, 'machine': 55, 'major': 56, 'making': 57, 'many': 58, 'may': 59, 'mean': 60, 'meaning': 61, 'measure': 62, 'metalearning': 63, 'method': 64, 'object':

In [49]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

In [50]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.show_topics(num_words=10)
for n, topic in topics:
    topic = topic.split('"')
    print(topic[1::2])

['method', 'model', 'time', 'data', 'function', 'analysis', 'causal', 'study', 'ha', 'treatment']
['model', 'use', 'time', 'test', 'data', 'using', 'instrument', 'system', 'molecular', 'tension']
['model', 'treatment', 'data', 'parameter', 'neto2', 'propose', 'local', 'survey', 'use', 'vulnerability']
['method', 'rings', 'use', 'property', 'structure', 'cluster', 'study', 'provide', 'algebra', 'result']
['use', 'model', 'treatment', 'time', 'oud', 'opioid', 'study', 'provide', 'receptor', 'ha']
['galaxy', 'lensing', 'dark', 'method', 'matter', 'weak', 'type', 'forklens', 'test', 'model']
['model', 'property', 'function', 'structure', 'sample', 'data', 'propose', 'generate', 'redshift', 'galaxy']
['survey', 'different', 'data', 'system', 'use', 'model', 'sample', 'approach', 'galaxy', 'propose']
['model', 'ideal', 'show', 'study', 'data', 'causal', 'method', 'propose', 'matter', 'abstraction']
['model', 'result', 'study', 'also', 'method', 'label', 'ring', 'system', 'lycopene', 'show']


In [51]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(7, 0.9935223)]
document ID 1:[(0, 0.59097964), (3, 0.39301482)]
document ID 2:[(0, 0.960984), (5, 0.033297174)]
document ID 3:[(5, 0.9927388)]
document ID 4:[(7, 0.9936122)]
document ID 5:[(1, 0.59698814), (7, 0.39763778)]
document ID 6:[(9, 0.990215)]
document ID 7:[(7, 0.97999454)]
document ID 8:[(0, 0.9926199)]
document ID 9:[(6, 0.9915065)]


In [52]:
categories = [dict["field"] for dict in corpus_paper]

In [53]:

for n in range(10):

    n = n*10+5
    #n番目のdocumentのトピック分布
    print(ldamodel.get_document_topics(corpus_[n])) 
    #print(categories[n])

    #print(" ".join(corpus_paper[n]["abst"]))


[(1, 0.59670335), (7, 0.39792264)]
[(8, 0.96714723), (9, 0.027255401)]
[(0, 0.98524225)]
[(0, 0.8863637), (1, 0.102362566)]
[(3, 0.9678474)]
[(3, 0.23353711), (8, 0.75217193)]
[(0, 0.13658276), (2, 0.8057209), (6, 0.051907066)]
[(1, 0.9930203)]
[(2, 0.9902146)]
[(6, 0.9912574)]


In [54]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [55]:
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [56]:
pyLDAvis.save_html(lda_display,'vis.html')